In [1]:
import xarray as xr
#import numpy as np
#import pandas as pd

#import os.path
#from datetime import datetime, timedelta, date
#import time

import matplotlib.pyplot as plt
from nmme_utils import *

In [2]:
# Eliminate Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Set xarray to keep attributes
xr.set_options(keep_attrs=True)  

In [4]:
def is_month(month):
   return (month == 6)

## Models and Forecast Settings

In [5]:
models_list,_,_,_=initModels()

## File Paths

In [6]:
url='http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME'

### Make a nmme_fcst `xarray.Dataset` containing all models + MME for months 1-9

In [7]:
print('PROCESSING CLIMOS FOR: ')

# Loop over all the Models
for imodel,nmme_model in enumerate(models_list):
    
    # Get the model, group, variables, and levels from the dictionary 
    varnames=nmme_model['varnames']
    levstrs=nmme_model['levstrs']
    model=nmme_model['model']
    group=nmme_model['group']
    
    print('===> ',model)
    
    # Loop over variables for this model
    for varname,levstr in zip(varnames,levstrs):
        
        print(varname,levstr)
        ofname='/mcs/scratch/kpegion/nmme/climatology/monthly/1991-2020/'+model+'.'+varname+'_'+levstr+'.clim.1991-2020.nc'

        # Read Data
        if group == "":
            if (model=='COLA-RSMAS-CCSM4'):
                hcstURL=url+'/.'+model+'/.MONTHLY'+'/.'+varname 
                fcstURL=url+'/.'+model+'/.MONTHLY'+'/.'+varname
            elif (model=='NCEP-CFSv2'):
                hcstURL=url+'/.'+model+'/.HINDCAST/.PENTAD_SAMPLES/.MONTHLY/.'+varname 
                fcstURL=url+'/.'+model+'/.FORECAST/.EARLY_MONTH_SAMPLES/.MONTHLY/.'+varname
            else:
                hcstURL=url+'/.'+model+'/.HINDCAST/.MONTHLY'+'/.'+varname
                fcstURL=url+'/.'+model+'/.FORECAST/.MONTHLY'+'/.'+varname
        else:
            hcstURL=url+'/.'+group+'/.'+model+'/.HINDCAST/.MONTHLY'+'/.'+varname
            fcstURL=url+'/.'+group+'/.'+model+'/.FORECAST/.MONTHLY'+'/.'+varname
         
        # Get Subsetted Data Using Ingrid
        print("Calling getClimDataViaIngrid")
        ds,ds_hcst,ds_fcst=getClimDataViaIngrid(hcstURL,fcstURL)
        print("Returning getClimDataViaIngrid")

        # Rename and clean up coordinates
        print("SETTING COORDS")
        ds['L'] = (ds['L'] - 0.5).astype('int')
        ds=ds.rename({'X':'lon','Y':'lat','L':'lead'})  
        ds['lead'].attrs = {'units': 'months'}
        ds['lon'].attrs['units']='degrees_east'
        ds['lat'].attrs['units']='degrees_north'
        print(ds)
        #if ('Z' in ds_clim.dims):
        #    ds_clim=ds_clim.drop('Z')
        
        # Calculate Climos
        print("Calcualting CLIM")
        ds_clim=ds.groupby('S.month').mean()
        print("DONE Calculating CLIM")
        
        print("DONE SETTING COORDS")
        print(ofname)
        print(ds_clim)
        
        print("WRITING CLIMATOLOGY")
        ds_clim.to_netcdf(ofname)
 

PROCESSING CLIMOS FOR: 
===>  NCEP-CFSv2
sst sfc
Calling getClimDataViaIngrid
http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.FORECAST/.EARLY_MONTH_SAMPLES/.MONTHLY/.sst/[M]average/dods
Returning getClimDataViaIngrid
SETTING COORDS
<xarray.Dataset>
Dimensions:  (lat: 181, lead: 10, S: 360, lon: 360)
Coordinates:
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lead     (lead) int64 0 1 2 3 4 5 6 7 8 9
  * S        (S) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-11-01 2020-12-01
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    sst      (S, lead, lat, lon) float64 dask.array<chunksize=(82, 10, 119, 119), meta=np.ndarray>
Attributes:
    Conventions:  IRIDL
Calcualting CLIM
DONE Calculating CLIM
DONE SETTING COORDS
/mcs/scratch/kpegion/nmme/climatology/monthly/1991-2020/NCEP-CFSv2.sst_sfc.clim.1991-2020.nc
<xarray.Dataset>
Dimensions:  (lat: 181, lead: 10, lon: 360, month: 12)
Coord